# Safety Bloods Date & Time Reconciliation

## Link Google Drive
Authorise application access to Google Drive (read & write permission).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "drive/My Drive/Colab Notebooks"

In [ ]:
%ls -l

Import external modules.

In [ ]:
import csv
import json
import pprint
import datetime
import os
import re
from matplotlib import pyplot

%matplotlib inline

## Setup Output Folder

_Global_ scope variables:

In [2]:
outputfolder = './test/safety/output/coh2'

Create the __output folder__ in which all output files will be placed.

In [3]:
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder, 0o755) # owner execution 

View contents of folder:

In [4]:
%ls -l ./test

total 0
drwxr-xr-x   9 herbsca  staff  288 17 Apr 11:48 pk/
drwxr-xr-x  10 herbsca  staff  320 20 Apr 12:22 safety/


Define output file writing functions:

In [5]:
def writeFile(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(pprint.pformat(contents))
    print('Text file write complete.')

def writeJSON(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(json.dumps(contents))
    print('JSON file write complete.')

def writeErrorCSV(filename: str, contents: object):
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        fieldnames = ['subjectid', 'type', 'collection', 'visit', 'panel', 'sourceDate', 'sourceTime', 'comparisonDate', 'comparisonTime', 'message']
        writer = csv.DictWriter(outputfile, fieldnames=fieldnames)
        writer.writeheader() # place headers in csv file.
        for key in contents:
            if key == 'errors' or  key  == 'total':
                    continue
            for data in contents[key]:
                # cycle through list of errors
                sourcedate = '-'
                sourcetime = '-'
                comparisondate = '-'
                comparisontime = '-'
                msg =  ''
                if 'date' in data and data['date']['error']:
                    sourcedate = data['date']['source']
                    comparisondate = data['date']['comparison']
                if 'time' in data and data['time']['error']:
                    sourcetime = data['time']['source']
                    comparisontime = data['time']['comparison']
                if 'msg' in data:
                    msg = data['msg']

                row = {
                    'subjectid': data['subject'],
                    'visit': data['visit'],
                    'type': data['type'],
                    'collection': data['collection'],
                    'panel': data['panel'],
                    'sourceDate': sourcedate,
                    'sourceTime': sourcetime,
                    'comparisonDate': comparisondate,
                    'comparisonTime': comparisontime,
                    'message': msg
                }
                writer.writerow(row) # write to file
    print('CSV file write complete.')

## Read Configuration File

Parse config file and establish relationships.

In [ ]:
# [TODO] - Specify the file path to the config json file
config_filename = './test/safety/config.json'

configDict = dict()

with open(config_filename) as jsonfile:
    configDict = json.load(jsonfile)

pprint.pp(configDict)

## Read Medrio Source File

Create dictionary mapping for each subject to allow for hash map search of data.

Context variables:

In [ ]:
# [TODO] - Specify the file path to the source .csv file
source_filename = './test/safety/source.csv'

# [TODO] - Specify the exact string column header for the subject id's
subjectidcol: str  = 'Subject ID'
# [TODO] - Specify the exact string column header for the visit name column
visitCol: str = 'Visit'

# REGULAR EXPRESSION OBJECTS
# [TODO] - Regex pattern to identify a date-time variable / data point
datetimeregex = re.compile(r'dattim', flags=re.I)
# [TODO] - Regex pattern to identify a Coagulation collection variable
coagregex = re.compile(r'coag', flags=re.I)
# [TODO] - Regex pattern to identify Chemistry collection variable
chemregex = re.compile(r'chem', flags=re.I)
# [TODO] - Regex pattern to identify Glucose collection variable
glucregex = re.compile(r'gluc', flags=re.I)
# [TODO] - Regex pattern to identify Serology collection variable
serregex = re.compile(r'ser', flags=re.I)
# [TODO] - Regex pattern to identify Haematology collection variable
haemregex = re.compile(r'haem', flags=re.I)
# [TODO] - Regex pattern to identify Urine collection variable
urineregex = re.compile(r'UC')
# [TODO] - Regex pattern to identify Comments collection variables
commregex = re.compile(r'comm', flags=re.I)

# ... insert more as necessary and attempt to add in the code in the following cell

Parse the source file and structure data into the __sourceMap__ dictionary variable.

In [ ]:
sourceMap = dict() # DICTIONARY FOR SUBJECT DATA

# READ THE SOURCE FILE
with open(source_filename) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # GET THE VISIT FOR THE ROW OF DATA
        try:
            visitid = configDict['match']['visit'][f'{row[visitCol]}']
        except:
            print(f'Error: Visit could not be matched from configuration file: {row[visitCol]}')
            continue
        
        # CREATE DICITONARY OBJECTS IF THEY DO NOT EXIST
        if row[subjectidcol] not in sourceMap:
            sourceMap[f'{row[subjectidcol]}'] = dict()
        if visitid not in sourceMap[f'{row[subjectidcol]}']:
            sourceMap[f'{row[subjectidcol]}'][visitid] = dict()
        
        # CYCLE THROUGH COLUMNS AND RECORD DATA VALUES
        for key in row:
            t = '' # TREAT AS THE "TYPE" VARIABLE
            if row[key] == '':
                continue # SKIP EMPTY CELLS
            value = dict()
            
            # SEARCH FOR URINE DATA POINT MATCH
            if urineregex.search(key) is not None:
                t = 'urine'
                if t not in sourceMap[f'{row[subjectidcol]}'][visitid]:
                    sourceMap[f'{row[subjectidcol]}'][visitid][f'{t}'] = dict()
                
                if datetimeregex.search(key) is not None:
                    # DATE AND TIME
                    valueError = False
                    try:
                        value = {'datetime': datetime.datetime.strptime(row.get(key).strip(), '%m/%d/%y %H:%M')} # EXTRACT & FORMAT DATE-TIME VARIABLE IN A CERTAIN WAY
                    except ValueError:
                        valueError = True
                    if valueError:
                        # try another date & time format
                        value = {'datetime': datetime.datetime.strptime(row.get(key).strip(), '%m/%d/%Y %H:%M')} # EXTRACT & FORMAT DATE-TIME VARIABLE IN A CERTAIN WAY
                else:
                    value = {'urine': row.get(key, 'No')} # RECORD IF URINE WAS COLLECTED OR NOT
            
            # SEARCH FOR BLOOD COLLECTION MATCH
            else:
                t = 'blood'
                if t not in sourceMap[f'{row[subjectidcol]}'][visitid]:
                    sourceMap[f'{row[subjectidcol]}'][visitid][f'{t}'] = dict()
                
                if datetimeregex.search(key) is not None:
                    valueError = False
                    try:
                        value = {'datetime': datetime.datetime.strptime(row.get(key).strip(), '%m/%d/%y %H:%M')} # EXTRACT & FORMAT DATE-TIME VARIABLE IN A CERTAIN WAY
                    except ValueError:
                        valueError = True
                    if valueError:
                        # try another date & time format
                        value = {'datetime': datetime.datetime.strptime(row.get(key).strip(), '%m/%d/%Y %H:%M')} # EXTRACT & FORMAT DATE-TIME VARIABLE IN A CERTAIN WAY
                    k = 'datetime'
                    
                # NOTE IF THE CERTAIN BLOODS WERE COLLECTED ? OR NOT ?
                elif coagregex.search(key) is not None:
                    k = 'coag'
                    value = {'coag': row.get(key, 'No')}
                elif chemregex.search(key) is not None:
                    k = 'chem'
                    value = {'chem': row.get(key, 'No')}
                elif glucregex.search(key) is not None:
                    k = 'gluc'
                    value = {'gluc': row.get(key, 'No')}
                elif serregex.search(key) is not None:
                    k = 'ser'
                    value = {'ser': row.get(key, 'No')}
                elif haemregex.search(key) is not None:
                    k = 'haem'
                    value = {'haem': row.get(key, 'No')}
                elif commregex.search(key) is not None:
                    k = 'comm'
                    value = {'comm': row.get(key, '')}
                else:
                    k = key
                    value = {'other': row.get(key, '')}
            
            sourceMap[f'{row[subjectidcol]}'][visitid][f'{t}'].update({**value}) # ADD THE ACCUMULATED VALUE TO THE SOURCE MAP

writeFile('sourceData.txt', sourceMap)

## Read Comparison File

Context variables:

In [ ]:
# [TODO] - Specify the file path to the comparison .csv file
comparison_filename = './test/safety/comparison_coh1&2.csv'

# REGULAR EXPRESSION OBJECTS
# [TODO] - Regex pattern to identify a date-time variable / data point
datetimeregex = re.compile(r'dattim', flags=re.I)
# [TODO] - Regex pattern to identify a Coagulation collection variable
coagregex = re.compile(r'coag', flags=re.I)
# [TODO] - Regex pattern to identify Chemistry collection variable
chemregex = re.compile(r'chem', flags=re.I)
# [TODO] - Regex pattern to identify Glucose collection variable
# Note: Only compared / searched for if "TestPanel" is left bank in the comparison file
glucregex = re.compile(r'gluc', flags=re.I)
# [TODO] - Regex pattern to identify Serology collection variable
serregex = re.compile(r'ser', flags=re.I)
# [TODO] - Regex pattern to identify Haematology collection variable
haemregex = re.compile(r'haem', flags=re.I)
# [TODO] - Regex pattern to identify Urine collection variable
urineregex = re.compile(r'ur', flags=re.I)
# [TODO] - Regex pattern to identify Comments collection variables
commregex = re.compile(r'comm', flags=re.I)

# [TODO] = Specify the exact column headers for these data points
subjectCol1 = 'PatientID1'
subjectCol2 = 'PatientID2'
dateCol = 'CollDate'
timeCol = 'CollTime'
visitCol = 'Visit'
panelCol = 'TestPanel'
nameCol = 'ResultName'

Parse comparison file data and structure into the __comparisonMap__ dictionary

In [ ]:
comparisonMap = dict() # EMPTY DICTIONARY

# READ COMPARISON FILE
with open(comparison_filename) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row[dateCol] == '':
            continue # SKIP EMPTY ROWS
        
        # INTERPRET VISIT
        try:
            visitid = configDict['match']['visit'][f'{row[visitCol]}']
        except:
            print(f'Error: Visit could not be matched from configuration file: {row[visitCol]}')
            continue
        
        subject = f'{row[subjectCol1]}' # GET THE SUBJECT ID (MEDRIO)
        
        if visitid == 6:
            continue # SKIP UNSCHEDULED VISIT
            
        # ADD DICTIONARY TO SOURCE MAP IF IT DOESN'T ALREADY EXIST
        if subject not in comparisonMap:
            comparisonMap[subject] = dict()
        if visitid not in comparisonMap[subject]:
            comparisonMap[subject][visitid] = dict()
        
        # FETCH INFORMATION IN ROW
        info = dict()
        
        # TIME & DATE COLLECTION INFORMATION
        timeval = {
            'date': datetime.datetime.strptime(row.get(dateCol).strip(), '%d/%m/%y'), # [TODO] - CHANGE FORMAT OF DATE VALUE READ - EXTRACT DATA VALUE AND PARSE BASED ON SPECIFIED FORMAT
            'time': datetime.datetime.strptime(row.get(timeCol).strip(), '%H:%M:%S') # [TODO] - CHANGE THE FORMAT OF THE TIME VALUE READ
        }
        
        k = None # TEST NAME
        
        # MATCH THE TEST PANEL TO A PARTICULAR DATA POINT COLLECTION
        if coagregex.search(row[panelCol]) is not None:
            k = 'coag'
            info[k] = {**timeval}
        elif chemregex.search(row[panelCol]) is not None:
            k = 'chem'
            info[k] = {**timeval}
        elif serregex.search(row[panelCol]) is not None:
            k = 'ser'
            info[k] = {**timeval}
        elif haemregex.search(row[panelCol]) is not None:
            k = 'haem'
            info[k] = {**timeval}
        elif urineregex.search(row[panelCol]) is not None:
            k = 'urine'
            info[k] = {**timeval}
        elif row[panelCol] == '':
            if glucregex.search(row[nameCol]) is not None:
                k = 'gluc'
                info[k] = {**timeval}
            else:
                continue # NOT INTERESTED - UNKNOWN COLLECTION
        else:
            continue # NOT INTERESTED - UNKNOWN COLLECTION
        
        # APPEND NEW INFORMATION ABOUT TEST PANEL IF NOT ALREADY DEFINED
        if k not in comparisonMap[subject][visitid]:
            comparisonMap[subject][visitid].update({**info}) # REMEMBER VALUES
        
        # SEE IF DATE IN COMPARISON FILE MATCHES PREVIOUSLY RECORDED DATE
        elif comparisonMap[subject][visitid][k]['date'] != info[k]['date']:
            # the date in the script does not match previously recorded dates
            error = {
                'subject': subject,
                'visit': row[visitCol],
                **info
            }
            pprint.pp(error)
            pprint.pp(comparisonMap[subject][visitid])
            raise Exception("ERROR: date for data point is not consistent within file!")
        
        # SEE IF TIME IN COMPARISON FILE MATCHES PREVIOUSLY RECORDED TIME
        elif comparisonMap[subject][visitid][k]['time'] != info[k]['time']:
            # the time in the script does not match previously recorded dates
            error = {
                'subject': subject,
                'visit': row[visitCol],
                **info
            }
            pprint.pp(error)
            pprint.pp(comparisonMap[subject][visitid])
            raise Exception("ERROR: time for data point is not consistent within file!")
        else:
            # APPEND INFORMATION ( MOST LIKELY JUST THE DATE AND TIME FOR THE ROW / TIMEPOINT)
            comparisonMap[subject][visitid].update(info)            

writeFile('comparisonData.txt', comparisonMap)

## Parse Data & Assess Equality

Context variables:

Compare matching data points and flag inequalities:

In [ ]:
outputLog = {"errors": 0, "total": 0}

# [TODO] - Specify the regex pattern to indentify urine data in blood samples
urineregex = re.compile(r'urine', flags=re.I)

subjectKeys = sourceMap.keys().__str__() # ALLOWS FOR EXTRACTION OF MEDRIO SUBJECT ID's

# CYCLE THROUGH EACH SUBJECT IN THE COMPARISON FILE
for s in comparisonMap:
    
    # REGEX PATTERN TO MATCH THE SUBJECT ID BY INSERTING THE SCREENING NUMBER -> "s"
    pattern = f'{s}-([0-9]){{5}}'
    match = re.search(pattern, subjectKeys) # FIND MEDRIO SUBJECT ID USING THE SCREENING NUMBER
    if match is None:
        print(f'{pattern} not in source.')
        continue # SUBJECT ID NOT IN SOURCE
    
    subject = match.group(0) # MATCHED RANDOMISATION NUMBER
    
    # STATUS REPORTING ON RANDOMISED SUBJECT
    outputLog[f'{subject}'] = list()

    # COMPARE DATA AND LOG ANY OUTLIERS / ERRORS
    # CYCLE THROUGH EACH VISIT IN THE COMPARISON FILE
    for visitid in comparisonMap[s]:
        # CYCLE THROUGH EACH TEST IN THE COMPARISON FILE
        for panel in comparisonMap[s][visitid]:
            # EXTRACT VISIT NAME USING INDEX INTO "_array" IN THE CONFIG FILE
            visitName = configDict['match']['visit']['_array'][visitid]
            
            # INITIAL VALUES
            errorObject = dict()
            datecmp = False
            timecmp = False
            t = '' # TYPE OF SAFETY COLLECTION (BLOOD OR URINE)
            
            # DETERMINE IF PANEL IS A URINE COLLECTION PANEL
            if urineregex.match(panel) is not None:
                t = 'urine'
            else:
                t = 'blood'
            
            # CYCLE THROUGH DATE AND TIME VARIABLES TO BE COMPARED
            try:
                if visitid not in sourceMap[subject]:
                    raise Exception(f'Visit not defined for the subject in source: {visitName}')
                if sourceMap[subject][visitid][t].get(panel, None) is not None and sourceMap[subject][visitid][t].get(panel, '') == 'Yes':
                    datecmp = sourceMap[subject][visitid][t]['datetime'].date() == comparisonMap[s][visitid][panel]['date'].date() # COMPARE SOURCE & COMPARISON DATE VALUES
                    timecmp = sourceMap[subject][visitid][t]['datetime'].time() == comparisonMap[s][visitid][panel]['time'].time() # COMPARE SOURCE & COMPARISON TIME VALUES
                else:
                    raise Exception('Test panel is not registed in the source data')

                if not datecmp:
                    # DATE VALUES ARE NOT EQUIVALENT
                    dateError = {
                            "source": sourceMap[subject][visitid][t]['datetime'].date().isoformat(),
                            "comparison": comparisonMap[s][visitid][panel]['date'].date().isoformat(),
                            "error": True
                        }
                    errorObject = {
                        "error": True,
                        "type": "ERROR",
                        "date": dateError
                    }

                if not timecmp:
                    # TIME VALUES ARE NOT EQUIVALENT
                    timeError = {
                            "source": sourceMap[subject][visitid][t]['datetime'].time().isoformat(),
                            "comparison": comparisonMap[s][visitid][panel]['time'].time().isoformat(),
                            "error": True
                        }
                    errorObject = {
                        "error": True,
                        "type": "ERROR",
                        "time": timeError
                    }
                if 'comm' in sourceMap[subject][visitid][t]:
                    # COMMENTS WHERE RECORDED HENCE JUST FLAG THEM FOR CHECKING
                    errorObject = {
                        "error": True,
                        "type": "WARNING",
                        "msg": sourceMap[subject][visitid][t]['comm'],
                        "date": {
                            "source": sourceMap[subject][visitid][t]['datetime'].date().isoformat(),
                            "comparison": comparisonMap[s][visitid][panel]['date'].date().isoformat(),
                            "error": True
                        },
                        "time": {
                            "source": sourceMap[subject][visitid][t]['datetime'].time().isoformat(),
                            "comparison": comparisonMap[s][visitid][panel]['time'].time().isoformat(),
                            "error": True
                        }
                    }
            
            # LOG ANY ERRORS TO THE ERROR FILE
            except KeyError as e:
                print(e)
                errorObject = {
                    "type": "WARNING",
                    "error": True,
                    'msg': "Warning: variable most likely not defined for subject in source."
                }
            except Exception as e:
                errorObject = {
                        "error": True,
                        "type": "WARNING",
                        "msg": f'{e}',
                        "panel": panel,
                        "visit": visitName,
                    }
            except:
                print('Unknown Error Occured.')
                errorObject = {
                    "error": True,
                    "type": "ERROR",
                    'msg': "Unknown error occured",
                }
            finally:
                if errorObject.get('error', False):
                    outputLog['errors'] += 1
                    outputLog[f'{subject}'].append({
                        **errorObject,
                        'visit': visitName,
                        'panel': panel,
                        'subject': subject,
                        'collection': t
                    })
                outputLog['total'] += 1 # INCREMENT NUMBER OF VARIABLES ASSESSED

# WRITE DICTINARY TO FILES
writeJSON('output.json', outputLog)
writeFile('output.txt', outputLog)
writeErrorCSV('output.csv', outputLog)

## Plotting Results

In [ ]:
pyplot.pie([outputLog['errors'], outputLog['total']], labels=[outputLog['errors'], outputLog['total']], colors=['#f42613', '#47f747'])